In [ ]:
import boto3
import getpass
import numpy as np
from preconfig import Preconfig


# 1. Upload config files to S3

In [ ]:
# Preconfig class allows us to parse a template file and generate a list of config files.
# Two loops puts the generated config files for a given number of repeats in S3.
preconfig = Preconfig()
path_to_template = '../templates/vary_compress_rate.cym.tpl'
configs = preconfig.parse(path_to_template,{})
s3_client = boto3.client("s3")
bucket = 'cytosim-working-bucket'
num_repeats = 5
job_names = []
buffered = np.empty((len(configs)), dtype=object)
for index, config in enumerate(configs):
    job_name = config[:-4]
    job_names.append(job_name)
    for repeat in range(num_repeats):
        opened_config = open(config, "rb")
        config_name = f'{job_name}/config/{job_name}_{repeat}.cym' 
        s3_client.put_object(Bucket=bucket, Key=config_name, Body=opened_config)
            
    
job_names

# 2a. Specify job definition

In [ ]:
job_definition_arn = "job_definition_arn"

# 2b. Create and register job definition

In [ ]:
from container_collection.batch.register_batch_job import register_batch_job
#from container_collection.batch.make_batch_job import make_batch_job

In [ ]:
job_definition_name = "karthikv_cytosim_varycompressrate"
image = "simularium/cytosim:latest"
vcpus = 1
memory = 7000
bucket_name = "s3://cytosim-working-bucket/"
simulation_name = ''

In [ ]:
account = getpass.getpass()

In [ ]:
def make_batch_job(
    name: str,
    image: str,
    account: str,
    region: str,
    user: str,
    vcpus: int,
    memory: int,
    prefix: str
) -> dict:
    return {
    "jobDefinitionName": f"{user}-{name}",
    "type": "container",
    "containerProperties": {
        "image": image,
        "vcpus": vcpus,
        "memory": memory,
        "environment": [
            {"name": "SIMULATION_TYPE", "value": "AWS"},
            {"name": "S3_INPUT_URL", "value": bucket_name},
            {"name": "SIMULATION_NAME", "value": simulation_name}
        ],
        "jobRoleArn": f"arn:aws:iam::{account}:role/BatchJobRole",
    },
}

In [ ]:
# Creating job definitions with make_batch_job
# Submitting job definitions with register_batch_job
jobs = np.empty(len(configs))
job_definitions = np.empty((len(configs)), dtype=object)
for index in range(len(configs)):
    print(index)
    simulation_name = job_names[index]
    print(simulation_name)
    job_definition = make_batch_job(f"cytosim-varycompressrate-dryrun-{str(index)}", 'simularium/cytosim:latest', account, 'us-west-2', 'karthikv', 1, 7000, 's3://cytosim-working-bucket/')
    registered_jd = register_batch_job(job_definition)
    job_definitions[index] = registered_jd
    # print(jobs[index])
    #print(jobs[index])
    

        


# 3. Submit job

In [ ]:
from container_collection.batch.submit_batch_job import submit_batch_job

In [ ]:
job_name = "cytosim-varycompressrate"
user = "karthikv"
queue = "general"
size = 5

In [ ]:
print(job_definitions)

In [ ]:
job_definitions

In [ ]:
for index in range(len(configs)):
    print(index)
    print(f'{job_name}-completerun-config{index}')
    submit_batch_job(name=f'{job_name}-completerun-config{index}', job_definition_arn=job_definitions[index],user=user,queue=queue,size=size)
    

In [ ]:
job_arns = submit_batch_job(
    name=job_name,
    job_definition_arn=job_definition_arn,
    user=user,
    queue=queue,
    size=size
)

In [ ]:
# For loop for multiple job submissions here

job_definition_arn

job_arns

# 4. Monitor job status

In [ ]:
# TODO: check job status, print progress bar

# 5. Load results